<a href="https://colab.research.google.com/github/aneeshraghav04/Comparison-of-Simple-Investment-Strategies/blob/main/value_momentum_nse_india.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Value and Momentum Strategies on Indian Indices (NSE/BSE)

This notebook implements two simple cross-sectional strategies on Indian large-cap stocks (for example, the NIFTY 50 constituents on NSE):

1. A value strategy based on valuation ratios.
2. A momentum strategy based on recent price performance.

Both strategies are constructed on the same Indian index universe and then compared on basic portfolio characteristics and overlap of holdings.

Data sources:

- Index constituents are loaded from the official Nifty 50 stock list CSV published by NSE Indices.
- Price history and fundamental ratios (P/E, P/B, P/S, Enterprise Value, EBITDA, Gross Profit) are retrieved via the `yfinance` Python library, which wraps Yahoo Finance data.


## Imports

In [1]:
import math
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

## Building the Indian large-cap universe

The universe in this notebook is the set of NIFTY 50 constituents from NSE.  
The list of current constituents is available as a CSV file from NSE Indices.  
Symbols are then mapped to Yahoo Finance tickers using the `.NS` suffix for NSE-listed stocks.


In [3]:
import requests
from io import StringIO

nifty50_url = "https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv"

session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/csv,application/octet-stream,application/vnd.ms-excel;q=0.9,*/*;q=0.8",
    "Referer": "https://www.niftyindices.com/"
}

response = session.get(nifty50_url, headers=headers, timeout=30)
response.raise_for_status()

nifty_raw = pd.read_csv(StringIO(response.text))
nifty = nifty_raw.rename(columns=str.strip)

if "Symbol" in nifty.columns:
    nifty["Ticker"] = nifty["Symbol"].astype(str).str.strip()
elif "SYMBOL" in nifty.columns:
    nifty["Ticker"] = nifty["SYMBOL"].astype(str).str.strip()
else:
    raise ValueError("Could not find a column named 'Symbol' or 'SYMBOL' in the Nifty 50 CSV.")

nifty["YF_Ticker"] = nifty["Ticker"].astype(str).str.strip() + ".NS"

nifty[["Ticker", "YF_Ticker"]].head()


,Ticker,YF_Ticker
0,ADANIENT,ADANIENT.NS
1,ADANIPORTS,ADANIPORTS.NS
2,APOLLOHOSP,APOLLOHOSP.NS
3,ASIANPAINT,ASIANPAINT.NS
4,AXISBANK,AXISBANK.NS


## Portfolio notional

A single notional portfolio size is used for both strategies and allocated equally across the selected stocks.


In [4]:
portfolio_size = 10_000_000.0
target_holdings = 30

## Helper functions

A small set of helpers standardises how fundamentals and price history are retrieved from Yahoo Finance for NSE tickers.


In [5]:
def get_ticker_info(yf_symbol):
    ticker = yf.Ticker(yf_symbol)
    try:
        info = ticker.get_info()
    except Exception:
        info = {}
    return info

def get_price_history(yf_symbol, max_days=400):
    ticker = yf.Ticker(yf_symbol)
    try:
        hist = ticker.history(period=f"{max_days}d")
    except Exception:
        hist = pd.DataFrame()
    return hist

## Value strategy on NSE stocks

The value strategy ranks stocks using a composite score built from:

- Trailing P/E ratio  
- Price-to-book ratio  
- Price-to-sales ratio (trailing twelve months)  
- Enterprise value to EBITDA  
- Enterprise value to gross profit  

Each ratio is converted into a cross-sectional percentile across the index universe, and the **Value Score** is defined as the average of these percentiles.  
Lower scores correspond to cheaper stocks on a relative basis.  

The strategy selects a fixed number of the cheapest stocks and assigns equal rupee weights to them.


In [6]:
value_rows = []

for _, row in nifty.iterrows():
    ticker = row["Ticker"]
    yf_symbol = row["YF_Ticker"]
    info = get_ticker_info(yf_symbol)
    price = info.get("currentPrice") or info.get("regularMarketPrice")
    market_cap = info.get("marketCap")
    pe = info.get("trailingPE")
    pb = info.get("priceToBook")
    ps = info.get("priceToSalesTrailing12Months")
    enterprise_value = info.get("enterpriseValue")
    ebitda = info.get("ebitda")
    gross_profit = info.get("grossProfits")

    ev_ebitda = np.nan
    if isinstance(enterprise_value, (int, float)) and isinstance(ebitda, (int, float)) and ebitda and ebitda > 0:
        ev_ebitda = enterprise_value / ebitda

    ev_gp = np.nan
    if isinstance(enterprise_value, (int, float)) and isinstance(gross_profit, (int, float)) and gross_profit and gross_profit > 0:
        ev_gp = enterprise_value / gross_profit

    value_rows.append(
        {
            "Ticker": ticker,
            "YF_Ticker": yf_symbol,
            "Price": price,
            "Market Cap": market_cap,
            "PE": pe,
            "PB": pb,
            "PS": ps,
            "EV/EBITDA": ev_ebitda,
            "EV/GP": ev_gp,
        }
    )

value_df = pd.DataFrame(value_rows)
value_df.head()

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DUMMYHDLVR.NS"}}}


,Ticker,YF_Ticker,Price,Market Cap,PE,PB,PS,EV/EBITDA,EV/GP
0,ADANIENT,ADANIENT.NS,2265.4,2.700160e+12,34.981470,5.083818,2.902649,28.891132,8.103715
1,ADANIPORTS,ADANIPORTS.NS,1509.4,3.260746e+12,27.240570,4.861755,9.384632,18.243909,16.340361
2,APOLLOHOSP,APOLLOHOSP.NS,7189.5,1.033740e+12,62.010520,11.367643,4.443383,35.048916,13.775040
3,ASIANPAINT,ASIANPAINT.NS,2968.5,2.845677e+12,73.278206,14.527259,8.288713,48.793052,19.430305
4,AXISBANK,AXISBANK.NS,1282.5,4.037160e+12,15.366643,1.993738,5.327610,NaN,7.286682


In [7]:
ratio_cols = ["PE", "PB", "PS", "EV/EBITDA", "EV/GP"]

filtered_value = value_df.copy()
for col in ratio_cols:
    filtered_value = filtered_value[filtered_value[col].notna()]
    filtered_value = filtered_value[filtered_value[col] > 0]

for col in ratio_cols:
    pct_col = col + " Percentile"
    filtered_value[pct_col] = filtered_value[col].rank(pct=True, ascending=True)

pct_cols = [c for c in filtered_value.columns if c.endswith("Percentile")]
filtered_value["Value Score"] = filtered_value[pct_cols].mean(axis=1)

filtered_value = filtered_value.sort_values("Value Score").reset_index(drop=True)

value_portfolio = filtered_value.head(min(target_holdings, len(filtered_value))).copy().reset_index(drop=True)

position_size_value = portfolio_size / len(value_portfolio)
value_portfolio["Number of Shares to Buy"] = (position_size_value / value_portfolio["Price"]).apply(
    lambda x: math.floor(x) if pd.notna(x) and x > 0 else 0
)

value_portfolio.head()

,Ticker,YF_Ticker,Price,Market Cap,PE,PB,PS,EV/EBITDA,EV/GP,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile,Value Score,Number of Shares to Buy
0,ONGC,ONGC.NS,241.23,3.035374e+12,8.283997,0.824467,0.462861,4.822605,3.675723,0.047619,0.023810,0.023810,0.023810,0.119048,0.047619,1381
1,TMPV,TMPV.NS,353.60,2.603160e+12,11.351525,1.176079,0.610389,5.261825,0.852717,0.095238,0.047619,0.047619,0.071429,0.023810,0.057143,942
2,HINDALCO,HINDALCO.NS,823.25,1.840069e+12,10.308665,1.355133,0.725665,6.898575,2.877910,0.071429,0.071429,0.071429,0.095238,0.095238,0.080952,404
3,COALINDIA,COALINDIA.NS,379.95,2.341529e+12,7.502963,2.220930,1.794202,4.861162,1.901657,0.023810,0.166667,0.190476,0.047619,0.047619,0.095238,877
4,NTPC,NTPC.NS,323.30,3.134932e+12,17.316550,1.631008,1.678640,10.440717,6.799904,0.119048,0.095238,0.166667,0.142857,0.285714,0.161905,1031


## Momentum strategy on NSE stocks

The momentum strategy focuses on recent price strength. For each stock, four lookback horizons are used:

- One-year price return  
- Six-month price return  
- Three-month price return  
- One-month price return  

Returns are computed from daily adjusted close prices retrieved from Yahoo Finance.  
Each horizon is converted into a cross-sectional percentile, and the **Momentum Score** is defined as the average of these percentiles.  
Higher scores correspond to stronger and more persistent momentum.

The strategy selects a fixed number of stocks with the highest Momentum Scores and assigns equal rupee weights to them.


In [8]:
horizon_map = {
    "One-Year Return": 252,
    "Six-Month Return": 126,
    "Three-Month Return": 63,
    "One-Month Return": 21,
}

momentum_rows = []

for _, row in nifty.iterrows():
    ticker = row["Ticker"]
    yf_symbol = row["YF_Ticker"]
    hist = get_price_history(yf_symbol, max_days=400)

    if hist.empty or "Close" not in hist.columns:
        momentum_rows.append(
            {
                "Ticker": ticker,
                "YF_Ticker": yf_symbol,
                "One-Year Return": np.nan,
                "Six-Month Return": np.nan,
                "Three-Month Return": np.nan,
                "One-Month Return": np.nan,
                "Price": np.nan,
                "Market Cap": np.nan,
            }
        )
        continue

    hist = hist.dropna(subset=["Close"])
    hist = hist.sort_index()

    latest_price = hist["Close"].iloc[-1]
    returns = {}

    for label, lag in horizon_map.items():
        if len(hist) > lag:
            past_price = hist["Close"].iloc[-lag]
            returns[label] = (latest_price / past_price) - 1.0 if past_price > 0 else np.nan
        else:
            returns[label] = np.nan

    ticker_info = get_ticker_info(yf_symbol)
    market_cap = ticker_info.get("marketCap")

    record = {
        "Ticker": ticker,
        "YF_Ticker": yf_symbol,
        "Price": latest_price,
        "Market Cap": market_cap,
    }
    record.update(returns)
    momentum_rows.append(record)

momentum_df = pd.DataFrame(momentum_rows)
momentum_df.head()

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: DUMMYHDLVR.NS"}}}
ERROR:yfinance:$DUMMYHDLVR.NS: possibly delisted; no price data found  (period=400d) (Yahoo error = "No data found, symbol may be delisted")


,Ticker,YF_Ticker,Price,Market Cap,One-Year Return,Six-Month Return,Three-Month Return,One-Month Return
0,ADANIENT,ADANIENT.NS,2265.399902,2.700160e+12,-0.091483,-0.122217,-0.019689,-0.043893
1,ADANIPORTS,ADANIPORTS.NS,1509.400024,3.260746e+12,0.194710,0.033838,0.119484,0.042187
2,APOLLOHOSP,APOLLOHOSP.NS,7189.500000,1.033740e+12,-0.003245,0.038474,-0.076671,-0.059212
3,ASIANPAINT,ASIANPAINT.NS,2968.500000,2.845677e+12,0.220014,0.334830,0.174780,0.137475
4,AXISBANK,AXISBANK.NS,1282.500000,4.037160e+12,0.107073,0.052214,0.215064,0.048822


In [9]:
return_cols = ["One-Year Return", "Six-Month Return", "Three-Month Return", "One-Month Return"]

filtered_mom = momentum_df.dropna(subset=return_cols).copy()

for col in return_cols:
    pct_col = col + " Percentile"
    filtered_mom[pct_col] = filtered_mom[col].rank(pct=True, ascending=True)

mom_pct_cols = [c for c in filtered_mom.columns if c.endswith("Percentile")]
filtered_mom["Momentum Score"] = filtered_mom[mom_pct_cols].mean(axis=1)

filtered_mom = filtered_mom.sort_values("Momentum Score", ascending=False).reset_index(drop=True)

momentum_portfolio = filtered_mom.head(min(target_holdings, len(filtered_mom))).copy().reset_index(drop=True)

position_size_mom = portfolio_size / len(momentum_portfolio)
momentum_portfolio["Number of Shares to Buy"] = (position_size_mom / momentum_portfolio["Price"]).apply(
    lambda x: math.floor(x) if pd.notna(x) and x > 0 else 0
)

momentum_portfolio.head()

,Ticker,YF_Ticker,Price,Market Cap,One-Year Return,Six-Month Return,Three-Month Return,One-Month Return,One-Year Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,Momentum Score,Number of Shares to Buy
0,ASIANPAINT,ASIANPAINT.NS,2968.500000,2.845677e+12,0.220014,0.334830,0.174780,0.137475,0.755102,0.979592,0.918367,1.000000,0.913265,112
1,SHRIRAMFIN,SHRIRAMFIN.NS,854.900024,1.608306e+12,0.385562,0.237776,0.441323,0.047222,0.918367,0.918367,1.000000,0.775510,0.903061,389
2,BHARTIARTL,BHARTIARTL.NS,2108.800049,1.284487e+13,0.342358,0.140115,0.117127,0.053768,0.897959,0.836735,0.775510,0.836735,0.836735,158
3,MARUTI,MARUTI.NS,16282.000000,5.119102e+12,0.478747,0.302384,0.067112,0.051877,0.959184,0.959184,0.591837,0.816327,0.831633,20
4,EICHERMOT,EICHERMOT.NS,7208.000000,1.977149e+12,0.522074,0.354380,0.057667,0.046762,0.979592,1.000000,0.551020,0.755102,0.821429,46


## Comparing value and momentum portfolios on NSE

With both strategies defined on the same NSE universe and notional portfolio size, their basic characteristics and overlap can be compared.


In [10]:
overlap = sorted(set(value_portfolio["Ticker"]) & set(momentum_portfolio["Ticker"]))
overlap_count = len(overlap)

comparison = pd.DataFrame(
    {
        "Strategy": ["Value", "Momentum"],
        "Number of Holdings": [len(value_portfolio), len(momentum_portfolio)],
        "Average Price": [value_portfolio["Price"].mean(), momentum_portfolio["Price"].mean()],
        "Median Market Cap": [value_portfolio["Market Cap"].median(), momentum_portfolio["Market Cap"].median()],
        "Average Score": [value_portfolio["Value Score"].mean(), momentum_portfolio["Momentum Score"].mean()],
        "Invested Rupees": [
            (value_portfolio["Price"] * value_portfolio["Number of Shares to Buy"]).sum(),
            (momentum_portfolio["Price"] * momentum_portfolio["Number of Shares to Buy"]).sum(),
        ],
    }
)

comparison

,Strategy,Number of Holdings,Average Price,Median Market Cap,Average Score,Invested Rupees
0,Value,30,2843.260,2.711262e+12,0.389524,9.955874e+06
1,Momentum,30,2583.702,3.304056e+12,0.656293,9.962446e+06


In [11]:
overlap_summary = pd.Series(
    {
        "Number of overlapping tickers": overlap_count,
        "Overlapping tickers": ", ".join(overlap),
    }
)

overlap_summary

,0
Number of overlapping tickers,16
Overlapping tickers,"ADANIPORTS, BAJAJ-AUTO, BAJAJFINSV, BHARTIARTL..."


### Interpreting the NSE-based strategies

Some simple ways to interpret the outputs:

- The comparison table summarises how the value and momentum portfolios differ in terms of average price, typical company size, average composite score, and rupee exposure.
- The overlap summary shows how many Indian large-cap stocks simultaneously look cheap on fundamentals and strong on momentum.
- The value strategy tilts toward lower valuation multiples within the NIFTY 50 universe, while the momentum strategy tilts toward recent winners.

To adapt this notebook to other Indian indices or exchanges:

- Replace the Nifty 50 CSV URL with the relevant index constituent file (for example, Nifty Next 50 or a BSE index list).
- For BSE symbols, use the `.BO` suffix for Yahoo Finance tickers instead of `.NS`.
